# 03. Timebanks: Cleaning & Feature Engineering
> Author: [Dawn Graham](https://dawngraham.github.io/)

Clean, create features, and combine datasets into single .csv for modeling.

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import regex as re
import time

import warnings
warnings.simplefilter(action='ignore')

from datetime import timedelta
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## Functions

In [2]:
# Import data
def open_data(data_set):
    data_set = pd.read_csv(f'../data/{str(data_set)}.csv')

    try:
        # Set to timeseries
        data_set['timestamp'] = pd.to_datetime(data_set['timestamp'])
    except:
        pass
    
    return data_set

## Cleaning & Feature Engineering

### Create `df` starting with `directory.csv`

In [3]:
df = open_data('directory')
df.head(3)

,address,country,focus,mission,name,notes,phone,postal,slug,sponsor,timebank,url,url_2,facebook,twitter
0,"7 Dickens St, Addington",New Zealand,NaN,Addington TimeBank: Serving Southern Christchu...,Addington TimeBank,This TimeBank became live on 1 February 2012. ...,64 3 3381613,"Christchurch, Canterbury 8024",timebanks/addington-timebank,Addington Community House (Inc),addington,http://addington.timebanks.org,NaN,https://www.facebook.com/Addingtontimebank/,NaN
1,"331 Sterling Highway, PO BOX 3493 (mailing add...",United States,NaN,Our mission is to strengthen our community b...,AHA Time Bank (Alaskans Helping Alaskans),NaN,907-299-2060,"Homer, Alaska 99603",timebanks/aha-time-bank-alaskans-helping-alaskans,NaN,aha,http://aha.timebanks.org,NaN,NaN,NaN
2,"Entrada al Refugio Tierra Colorada Baja , PO B...",Guatemala,NaN,Build adaptive capacity for integrated sustain...,Alticultura,NaN,1150230873516,"Quetzaltenango, Quetzaltenango 9001",timebanks/alticultura,Alticultura,alticultura,http://alticultura.timebanks.org,NaN,NaN,NaN


In [4]:
# Drop `focus` column - no timebanks have value for this
df.drop(columns='focus', inplace=True)

# Drop unnecessary columns
df.drop(columns=['name', 'slug', 'url'], inplace=True)

### Sentiment analysis
Reference: [Simplifying Sentiment Analysis using VADER in Python (on Social Media Text)](https://medium.com/analytics-vidhya/simplifying-social-media-sentiment-analysis-using-vader-in-python-f9e6ec6fc52f)

In [5]:
# Get compound sentiment score for mission statements and notes
analyser = SentimentIntensityAnalyzer()

df['mission_sentiment'] = ''
df['notes_sentiment'] = ''

for row in range(len(df)):
    try:
        df['mission_sentiment'][row] = analyser.polarity_scores(df['mission'][row])['compound']
    except:
        df['mission_sentiment'][row] = 0
    try:
        df['notes_sentiment'][row] = analyser.polarity_scores(df['notes'][row])['compound']
    except:
        df['notes_sentiment'][row] = 0

### Change to numerical data

In [6]:
# Get word count for mission statement and notes
df['mission_word_count'] = ''
df['notes_word_count'] = ''

for row in range(len(df)):
    try:
        df['mission_word_count'][row] = len(df['mission'][row].split())
    except:
        df['mission_word_count'][row] = 0
    try:
        df['notes_word_count'][row] = len(df['notes'][row].split())
    except:
        df['notes_word_count'][row] = 0

# Drop `mission` and `notes` columns
df.drop(columns=['mission', 'notes'], inplace=True)

In [7]:
# Change 5555555555 phone number to NaN
df['phone'][6] = np.nan

In [8]:
# Set columns to 1 if timebank has feature, 0 if not
df[['phone', 'sponsor', 'url_2', 'facebook', 'twitter']] = df[['phone', 'sponsor', 'url_2', 'facebook', 'twitter']].notnull().astype(int)

In [9]:
# Reorder columns, including only timebank and numerical
df = df[['timebank', 'sponsor', 'phone', 'url_2', 'facebook', 'twitter', 'mission_word_count', 'mission_sentiment', 'notes_word_count', 'notes_sentiment']]

In [10]:
df.head()

,timebank,sponsor,phone,url_2,facebook,twitter,mission_word_count,mission_sentiment,notes_word_count,notes_sentiment
0,addington,1,1,0,1,0,320,0.9986,144,0.8943
1,aha,0,1,0,0,0,20,0.7717,0,0
2,alticultura,1,1,0,0,0,12,0,0,0
3,andersoncommunity,1,1,0,0,0,20,0.6369,0,0
4,ate,1,1,1,1,0,11,0.4215,0,0


### Add info from `updates.csv`

In [11]:
updates = open_data('updates')
updates.head()

,exchanges,hours,last_exchange,members,offers,requests,timebank,timestamp
0,1133,5998,1 day 7 hours ago,179,36,27,addington,2019-01-24 09:35:07
1,439,1375,2 years 41 weeks ago,207,0,0,aha,2019-01-24 09:35:07
2,none (start up),none (start up),none (start up),2,0,0,alticultura,2019-01-24 09:35:07
3,2,3,3 years 12 weeks ago,8,0,0,andersoncommunity,2019-01-24 09:35:07
4,1,5,1 year 30 weeks ago,8,4,4,ate,2019-01-24 09:35:07


In [12]:
# Convert `last_exchange` to date, time
updates['last_exchange_time'] = ''

for i in range(len(updates)):
    try:
        year = int(re.search('(\d+)\syear', updates.iloc[i]['last_exchange']).group(1))
    except:
        year = 0
        
    try:
        week = int(re.search('(\d+)\sweek', updates.iloc[i]['last_exchange']).group(1))
    except:
        week = 0
        
    try:
        day = int(re.search('(\d+)\sday', updates.iloc[i]['last_exchange']).group(1))
    except:
        day = 0
        
    try:
        hour = int(re.search('(\d+)\shour', updates.iloc[i]['last_exchange']).group(1))
    except:
        hour = 0
        
    try:
        minute = int(re.search('(\d+)\smin', updates.iloc[i]['last_exchange']).group(1))
    except:
        minute = 0
        
    try:
        sec = int(re.search('(\d+)\ssec', updates.iloc[i]['last_exchange']).group(1))
    except:
        sec = 0
    
    start = updates['timestamp'][i]
    end = timedelta(weeks=year*365 + week, days=day, hours=hour, minutes=minute, seconds=sec)
    
    updates['last_exchange_time'][i] = start - end

# Change to datetime
updates['last_exchange_time'] = pd.to_datetime(updates['last_exchange_time'])

# Set `last_exchange_time` to nan if start up with no exchanges
updates.loc[updates['exchanges'] == 'none (start up)', 'last_exchange_time'] = np.nan

In [13]:
# Replace `none (start up)` with 0 for all columns
updates.replace('none (start up)', 0, inplace=True)

# Change to `int`
updates['exchanges'] = updates['exchanges'].astype(int)
updates['hours'] = updates['hours'].astype(int)

In [14]:
updates.head()

,exchanges,hours,last_exchange,members,offers,requests,timebank,timestamp,last_exchange_time
0,1133,5998,1 day 7 hours ago,179,36,27,addington,2019-01-24 09:35:07,2019-01-23 02:35:07
1,439,1375,2 years 41 weeks ago,207,0,0,aha,2019-01-24 09:35:07,2004-04-15 09:35:07
2,0,0,0,2,0,0,alticultura,2019-01-24 09:35:07,NaT
3,2,3,3 years 12 weeks ago,8,0,0,andersoncommunity,2019-01-24 09:35:07,1997-11-06 09:35:07
4,1,5,1 year 30 weeks ago,8,4,4,ate,2019-01-24 09:35:07,2011-06-30 09:35:07


**Daily Averages**

In [15]:
# Get total number of days covered in dataset
total_days = (updates['timestamp'].max() - updates['timestamp'].min()).round('D').days

# Get average number of daily exchanges and hours
df['avg_daily_exchanges'] = ''
df['avg_daily_hours'] = ''
df['hours_per_exchange'] = ''

for row in range(len(df)):
    
    # Get numbers at start and end of collection period
    start_exchanges = updates[updates['timebank'] == df['timebank'][row]]['exchanges'].min()
    end_exchanges = updates[updates['timebank'] == df['timebank'][row]]['exchanges'].max()
    
    start_hours = updates[updates['timebank'] == df['timebank'][row]]['hours'].min()
    end_hours = updates[updates['timebank'] == df['timebank'][row]]['hours'].max()
    
    # Divide difference by number of days
    df['avg_daily_exchanges'][row] = (end_exchanges - start_exchanges) / total_days
    df['avg_daily_hours'][row] = (end_hours - start_hours) / total_days
    
    # Get average hours per exchange
    if df['avg_daily_exchanges'][row] == 0:
        df['hours_per_exchange'][row] = 0
    else:
        df['hours_per_exchange'][row] = df['avg_daily_hours'][row] / df['avg_daily_exchanges'][row]

In [16]:
# Get average offer:request ratio and append to df
df['avg_offers'] = ''
df['avg_requests'] = ''
df['offer_request_ratio'] = ''

for row in range(len(df)):
    offers = updates[updates['timebank'] == df['timebank'][row]]['offers'].mean()
    requests = updates[updates['timebank'] == df['timebank'][row]]['requests'].mean()
    
    df['avg_offers'][row] = offers
    df['avg_requests'][row] = requests
    
    try:
        df['offer_request_ratio'][row] = offers / requests
    except:
        df['offer_request_ratio'][row] = 0

In [17]:
df['members_starting'] = ''
df['members_daily_new'] = ''

for row in range(len(df)):
    
    # Get member numbers at start and end of collection period
    start_members = updates[updates['timebank'] == df['timebank'][row]]['members'].iloc[0]
    end_members = updates[updates['timebank'] == df['timebank'][row]]['members'].iloc[-1]
    
    df['members_starting'][row] = start_members
    
    # Get average daily new members
    df['members_daily_new'][row] = (end_members - start_members) / total_days

In [18]:
df.head(15)

,timebank,sponsor,phone,url_2,facebook,twitter,mission_word_count,mission_sentiment,notes_word_count,notes_sentiment,avg_daily_exchanges,avg_daily_hours,hours_per_exchange,avg_offers,avg_requests,offer_request_ratio,members_starting,members_daily_new
0,addington,1,1,0,1,0,320,0.9986,144,0.8943,0.236842,1.76316,7.44444,37.3421,32.9474,1.13339,179,0.0263158
1,aha,0,1,0,0,0,20,0.7717,0,0,0,0,0,0,0,0,207,0
2,alticultura,1,1,0,0,0,12,0,0,0,0,0,0,0,0,0,2,0
3,andersoncommunity,1,1,0,0,0,20,0.6369,0,0,0,0,0,0,0,0,8,0
4,ate,1,1,1,1,0,11,0.4215,0,0,0,0,0,4,4,1,8,0
5,avl,0,1,0,0,0,8,0.6908,2,0,0,0,0,0,0,0,2,0
6,atx,0,0,0,1,0,53,0.7579,0,0,0,0,0,90,80.8158,1.11364,58,-0.0263158
7,labodunautilus,0,1,1,1,0,25,0,0,0,0,0,0,0,0,0,11,0
8,blackhills,0,1,0,0,0,35,0.765,0,0,0.0526316,0.421053,8,1.60526,2.63158,0.61,37,0.0526316
9,ujimaboston,0,1,0,1,0,6,0.6597,0,0,1.13158,135.605,119.837,61.9474,51.6053,1.20041,176,0.631579


### Add info from `social.csv`

In [19]:
social = open_data('social')
social.head(3)

,timestamp,timebank,facebook,facebook_likes,facebook_followers,twitter,twitter_joined_date,twitter_tweets,twitter_following,twitter_followers
0,2019-02-03 12:45:00,addington,https://www.facebook.com/Addingtontimebank/,169,172,NaN,NaN,NaN,NaN,NaN
1,2019-02-03 12:45:00,aha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-02-03 12:45:00,alticultura,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Replace missing values with 0
social.replace([np.nan, 'page unavailable', 'account suspended'], 0, inplace=True)

# Append social columns to df
df = pd.merge(df, social[['timebank', 'facebook_likes', 'facebook_followers',
                    'twitter_tweets', 'twitter_following', 'twitter_followers']], on='timebank')

### Add category info from `talents.csv`, `offers.csv`, and `requests.csv`

In [21]:
# Open category files and change timestamp to day only
talents = open_data('talents')
talents['timestamp'] = talents['timestamp'].dt.floor('D')

offers = open_data('offers')
offers['timestamp'] = offers['timestamp'].dt.floor('D')

requests = open_data('requests')
requests['timestamp'] = requests['timestamp'].dt.floor('D')

In [22]:
# Merge all into categories dataframe
merge_columns = ['cat_id', 'cat_parent', 'category', 'timebank', 'timestamp']
categories = talents.merge(offers, how='left', on=merge_columns)
categories = categories.merge(requests, how='left', on=merge_columns)

# Replace nan values with 0
categories.replace(np.nan, 0, inplace=True)

# Reorder columns for viewing
categories = categories[['timestamp', 'timebank', 'category', 'cat_id', 'cat_parent',
           'count_talent', 'count_offers', 'count_requests']]

# Get values of offers - requests
categories['offers_requests_diff'] = categories['count_offers'] - categories['count_requests']

In [23]:
# Save to .csv
categories.to_csv('../data/categories.csv', index=False)

In [24]:
df['categories_total'] = ''
df['categories_parent'] = ''
df['categories_with_offers'] = ''
df['categories_with_requests'] = ''

for row in range(len(df)):
    
    # Filter to specific timebank
    tb = categories[categories['timebank'] == df['timebank'][row]]

    # Get total number of categories
    total_cat = tb['cat_id'].nunique()
    df['categories_total'][row] = total_cat

    # Get number of parent categories
    df['categories_parent'][row] = tb[tb['cat_parent'] == 'is_parent']['cat_id'].nunique()
    
    # Get percent of categories with offers
    try:
        df['categories_with_offers'][row] = tb[tb['count_offers'] != 0]['cat_id'].nunique() / total_cat
    except:
        df['categories_with_offers'][row] = 0
    
    # Get percent of categories with requests
    try:
        df['categories_with_requests'][row] = tb[tb['count_requests'] != 0]['cat_id'].nunique() / total_cat
    except:
        df['categories_with_requests'][row] = 0

In [25]:
df['talent_per_cat_per_member'] = ''

for row in range(len(df)):
    
    # Filter to specific timebank
    tb = categories[categories['timebank'] == df['timebank'][row]]
    
    # Get average total talents for all parent categories
    catparent = tb[tb['cat_parent'] == 'is_parent']['count_talent'].groupby(tb['timestamp']).sum().mean()
    
    # Get total talents / number of parent categories / total members
    try:
        df['talent_per_cat_per_member'][row] = catparent / df['categories_parent'][row] / df['members_starting'][row]
    except:
        df['talent_per_cat_per_member'][row] = 0

In [26]:
# # Look at specific timebank
# categories[categories['timebank'] == 'addington'].sort_values('offers_requests_diff')

### Convert all numerical columns to `float`

In [27]:
df.loc[:, df.columns != 'timebank'] = df.loc[:, df.columns != 'timebank'].astype(float)

In [28]:
# Preview
df.head(10).T

,0,1,2,3,4,5,6,7,8,9
timebank,addington,aha,alticultura,andersoncommunity,ate,avl,atx,labodunautilus,blackhills,ujimaboston
sponsor,1,0,1,1,1,0,0,0,0,0
phone,1,1,1,1,1,1,0,1,1,1
url_2,0,0,0,0,1,0,0,1,0,0
facebook,1,0,0,0,1,0,1,1,0,1
twitter,0,0,0,0,0,0,0,0,0,0
mission_word_count,320,20,12,20,11,8,53,25,35,6
mission_sentiment,0.9986,0.7717,0,0.6369,0.4215,0.6908,0.7579,0,0.765,0.6597
notes_word_count,144,0,0,0,0,2,0,0,0,0
notes_sentiment,0.8943,0,0,0,0,0,0,0,0,0


### Export to `combined.csv`

In [29]:
# Include dates covered in file name
start_date = updates['timestamp'].min().strftime('%Y-%m-%d')
end_date = updates['timestamp'].max().strftime('%Y-%m-%d')

df.to_csv(f'../data/combined_{start_date}_{end_date}.csv', index=False)